import pyspark

spark = pyspark.sql.SparkSession.builder.appName("HM")\
        .config("spark.executor.memory", "70g")\
        .config("spark.driver.memory", "50g")\
        .getOrCreate()

trans_spark = spark.read.csv("transactions_train.csv")

trans_spark.head()

from pyspark.sql.functions import datediff, round, to_date, lit

%%time
w = trans_spark.select(round(
            datediff(trans_spark["_c0"], to_date(lit("2018-01-01")))
            / 7))\
    .collect()

In [1]:
import pandas as pd
import swifter
import scipy.sparse
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
%%time
trans = pd.read_csv("./transactions_train.csv", parse_dates=["t_dat"])

CPU times: user 16.3 s, sys: 2.08 s, total: 18.4 s
Wall time: 20.9 s


In [3]:
trans.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [4]:

t0 = pd.Timestamp("2018-1-1")
td = pd.Timedelta(weeks=1)
week = trans.t_dat.parallel_apply(lambda t: int((t-t0)/td))

In [5]:
trans["week"] = week

In [6]:
import joblib
import tqdm

tqdm.notebook.tqdm.pandas()

In [7]:

trans["lst_art"] = trans["article_id"].parallel_apply(lambda x: [x])

In [8]:
%%time
cust_week = trans.groupby(["customer_id", "week"]).progress_apply(lambda s: s["lst_art"].sum())

  0%|          | 0/8088422 [00:00<?, ?it/s]

CPU times: user 18min 47s, sys: 45.1 s, total: 19min 32s
Wall time: 22min 28s


In [11]:
cust_week.to_csv("./cust_week.csv")